In [7]:
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd
import numpy as np
import numba as nb
import imageio
import os
import shutil

randomSeed = 10

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(
    style="ticks", 
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['animation.ffmpeg_path'] = "/opt/conda/bin/ffmpeg"

from main import *
from multiprocessing import Pool
import pandas as pd

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])


from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [6]:
maps3 = pd.read_csv("L10SingleDistributeClusterDeltaOmega2.csv", index_col=0)  # clusterDeltaOmega2, deltaOmega1
maps3.index = np.round(maps3.index.astype(float), 4)
maps3.columns = np.round(maps3.columns.astype(float), 4)

maps = maps3
maps.columns = maps.columns.astype(np.float64)

globalRd = 1.2828

def lambda_c0(d0):
    rs = 1
    rd = globalRd
    if d0 + 1 < rd:
        si = 0
    else:
        beta = 2 * np.arccos((d0 ** 2 + rd ** 2 - rs ** 2) / (2 * d0 * rd))
        alpha = 2 * np.arccos((rd ** 2 + rs ** 2 - d0 ** 2) / (2 * rd * rs))
        si = alpha * rs ** 2 / 2 + beta * d0 ** 2 / 2 - d0 * rd * np.sin(beta / 2)
    Nc = 500
    # deltaOmega = 3 - 3 / 1.1  # 
    deltaOmega = 3 / (globalRd - 0.1) - 3 / globalRd

    NonR = Nc / 2 * (rd + d0 - 1)
    Sr1 = np.pi * ((rd + d0)**2 - 1**2)
    rho1 = NonR / Sr1
    sigmaA1 = (np.pi * d0 ** 2 - si) * rho1

    return 1 / sigmaA1 * deltaOmega

def lambda_c1(d0):
    rs = 3
    Nc = 500 / 2
    Sr = np.pi * (3**2 - 1**2)
    deltaOmega = 2 / Nc
    beta = 2 * np.arccos(1 - d0 ** 2 / (2 * rs ** 2))
    alpha = np.pi - beta / 2
    S1 = d0**2 * alpha / 2 + rs**2 * beta / 2 - rs * d0 * np.sin(alpha / 2)
    sigmaA = S1 * Nc / Sr
    return deltaOmega / sigmaA * 1

def lambda_c2(d0):
    rs = 1
    deltaOmega = 2
    beta = 2 * np.arccos(1 - d0 ** 2 / (2 * rs ** 2))
    alpha = np.pi - beta / 2
    N = 500
    return (
        np.pi * rs ** 2 * deltaOmega
        / (N - 1)
        / (alpha * d0**2 / 2 + beta * rs**2 / 2 - rs * d0 * np.sin(alpha / 2))
    )

def lambda_c3(d0):
    L = 10
    deltaOmega = 1
    N = 500
    return (
        deltaOmega
        / (N * np.pi * d0 ** 2 / L ** 2) / 2
    )

funcIdxs = dict()
for func in [lambda_c0, lambda_c1, lambda_c2, lambda_c3]:
    funcIdxs[func] = []
    for d0 in maps.columns:
        l = func(d0)
        idx = 0
        upper = maps.index[maps.index > l].min()
        if np.isnan(upper):
            upper = maps.index.max()
        idx = idx + np.where(maps.index == upper)[0][0]
        lower = maps.index[maps.index < l].max()
        if np.isnan(lower):
            lower = maps.index.min()
        idx = idx + np.where(maps.index == lower)[0][0]
        idx = idx / 2
        funcIdxs[func].append(idx)

line0 = pd.Series(funcIdxs[lambda_c0]).rolling(2, center=True, min_periods=1).mean()
line1 = pd.Series(funcIdxs[lambda_c1]).rolling(1, center=True, min_periods=1).mean()
line2 = pd.Series(funcIdxs[lambda_c2]).rolling(2, center=True, min_periods=1).mean()
line3 = pd.Series(funcIdxs[lambda_c3]).rolling(5, center=True, min_periods=1).mean()

rangeLambdas = np.concatenate([
    np.arange(0.01, 0.1, 0.005), np.arange(0.1, 0.31, 0.05)
])
distanceDs = np.concatenate([
    np.arange(0.1, 1.1, 0.05)
])
xIdxs = np.arange(len(distanceDs), step=6)
yIdxs = np.concatenate([np.arange(18, step=6), np.arange(18, 23, step=4)])

In [15]:
def plot_last_state(sa: StateAnalysis, ax: plt.Axes = None, index: int = -1,
                    showTicks: bool = False, tickSize: int = 16):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))

    singlePositionX, singlePhaseTheta, _ = sa.get_state(index=index)
    omegaTheta = sa.model.omegaTheta

    ax.quiver(
        singlePositionX[:, 0], singlePositionX[:, 1],
        np.cos(singlePhaseTheta[:]), np.sin(singlePhaseTheta[:]), 
        color=[cmap(i) for i in (omegaTheta - 1) / 2], alpha=1,
        scale=23, width=0.005
    )

    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    if showTicks:
        ax.set_xticks([0, 10])
        ax.set_yticks([0, 10])
        ax.set_xlabel(r"$x_i$", fontsize=tickSize)
        ax.set_ylabel(r"$y_i$", fontsize=tickSize, rotation=0)
        ax.set_xticklabels(ax.get_xticks(), fontsize=tickSize)
        ax.set_yticklabels(ax.get_yticks(), fontsize=tickSize)

        ax.xaxis.labelpad = -20
        ax.yaxis.labelpad = -10
    else:
        ax.set_xticks([])
        ax.set_yticks([])
    ax.grid(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['right'].set_color('black')
    plt.tick_params(direction='in', length=3)


def draw_phase_dia_single_dis(fileName: str, mp4Path: str = "./normalMP4", step: int = 1, tailCounts: int = None):

    def plot_frame(i):
        fig, ax = plt.subplots(1, 1, figsize=(5.1, 5))
        # phase diagram
        plt.plot(line0[:10], 
                linestyle="--", color="black", lw=2, zorder=2)
        plt.plot(line1[:4],
                linestyle="--", color="black", lw=2, zorder=2)
        plt.plot(np.arange(maps.shape[1]), line2,
                linestyle="--", color="black", lw=2, zorder = 2)
        plt.plot(np.arange(maps.shape[1]), line3,
                linestyle="--", color="black", lw=2, zorder = 2)
        plt.fill_between(
            np.arange(maps.shape[1]), 0, line1, alpha=0.8, color="#403990"
        )
        plt.fill_between(
            np.arange(maps.shape[1]), line1, line0, alpha=0.8, color="#80A6E2"
        )
        plt.fill_between(
            np.arange(maps.shape[1]), line0, line2, alpha=0.9, color="#FBDD85"
        )
        plt.fill_between(
            np.arange(maps.shape[1]), line2, line3, alpha=1, color="#F46F43"
        )
        plt.fill_between(
            np.arange(maps.shape[1]), line3, len(rangeLambdas) - 1, alpha=1, color="#CF3D3E"
        )

        ax.plot([-4, 0], [9, 0], clip_on=False, color="black", lw=1)  # Disorder
        ax.plot([-4, 3], [24, 5], clip_on=False, color="black", lw=1)  # ICS
        ax.plot([22, 18], [6, 0], clip_on=False, color="black", lw=1)  # CCS
        ax.plot([15, 22], [22, 24], clip_on=False, color="black", lw=1)

        plt.text(0.2, 0.3, "DS", fontsize=16, color="white")
        plt.text(1.5, 3, "ICS", fontsize=18, color="black")
        plt.text(5.8, 3, "CCS", fontsize=20, color="black")
        plt.text(9.5, 10, "ACLS", ha="center", va="center", fontsize=20, color="black")
        plt.text(15, 15.5, "ICLS", ha="center", va="center", fontsize=20, color="black")

        plt.xticks(xIdxs, np.round(distanceDs, 2)[xIdxs])
        plt.yticks(yIdxs, np.round(rangeLambdas, 2)[yIdxs])
        plt.tick_params(direction='in', length=3)

        plt.ylim(0, maps.shape[0] - 1)
        plt.xlim(0, maps.shape[1] - 1)

        plt.xlabel(r"$d_0$")
        plt.ylabel(r"$\lambda$", rotation=0)
        plt.grid(False)
        # snapshot
        axICS = inset_axes(ax, width="40%", height="40%", loc='lower left',
                    bbox_to_anchor=(-0.8, 0.5, 1.5, 1.5),
                    bbox_transform=ax.transAxes)

        plot_last_state(sas[0], ax=axICS, index=i)
        axICS.set_title(r"(b) $\lambda=0.035,\ d_0=0.25$")

        axDisorder = inset_axes(ax, width="40%", height="40%", loc='lower left',
                            bbox_to_anchor=(-0.8, -0.2, 1.5, 1.5),
                            bbox_transform=ax.transAxes)
        plot_last_state(sas[1], ax=axDisorder, index=i)
        axDisorder.set_title(r"(c) $\lambda=0.01,\ d_0=0.1$")

        axQuickSync = inset_axes(ax, width="40%", height="40%", loc='lower left',
                                bbox_to_anchor=(1.05, 0.5, 1.5, 1.5),
                                bbox_transform=ax.transAxes)
        plot_last_state(sas[2], ax=axQuickSync, index=i)
        axQuickSync.set_title(r"(d) $\lambda=0.3,\ d_0=0.8$")

        axDRCS = inset_axes(ax, width="40%", height="40%", loc='lower left',
                            bbox_to_anchor=(1.05, -0.2, 1.5, 1.5),
                            bbox_transform=ax.transAxes)
        plot_last_state(sas[3], ax=axDRCS, index=i)
        axDRCS.set_title(r"(e) $\lambda=0.01,\ d_0=1$")

        colorbarAx = inset_axes(ax, width="5%", height="100%", loc='center',
                                bbox_to_anchor=(1.25, 0, 1, 1),
                                bbox_transform=ax.transAxes)
        sca = ax.scatter(-np.ones_like(omegaTheta), -np.ones_like(omegaTheta), 
                c=np.linspace(1, 3, len(omegaTheta)), cmap=cmap, s=100)
        cbar = plt.colorbar(sca, cax=colorbarAx, ticks=[1, 2, 3])
        cbar.ax.set_ylim(1, 3)
        cbar.ax.set_yticklabels(['$1$', '$2$', '$3$'])
        cbar.set_label(r"$\omega$", fontsize=16, rotation=0)
        cbar.ax.yaxis.set_label_coords(-0.6, 0.5)
        cbar.ax.tick_params(direction='in', length=3)
        ax.set_title(r"(a)", loc="left", fontsize=18)

        plt.savefig(f"{mp4Path}/{fileName}/{i}.png", dpi=200, bbox_inches="tight")
        plt.close()

    models = [
        SingleDistribution(0.035, 0.25, 10, savePath="./data", randomSeed=randomSeed),
        SingleDistribution(0.01, 0.1, 10, savePath="./data", randomSeed=randomSeed),
        SingleDistribution(0.3, 0.8, 10, savePath="./data", randomSeed=randomSeed),
        SingleDistribution(0.01, 1, 10, savePath="./data", randomSeed=randomSeed)
    ]
    np.random.seed(10)
    omegaTheta = np.random.uniform(1, 3, size=500)

    sas = [StateAnalysis(model, showTqdm=False) for model in models]

    # fig, ax = plt.subplots(1, 1, figsize=(5.1, 5))
    TNum = sas[0].TNum
    startIdx = TNum - tailCounts * step if tailCounts else 0
    frames = np.arange(startIdx, TNum, step)
    pbar = tqdm(total=len(frames))
    for i in frames:
        plot_frame(i)
        pbar.update(1)
    pbar.close()

In [17]:
draw_phase_dia_single_dis("SingleDistribution", step=1, tailCounts=750)

  0%|          | 0/750 [00:00<?, ?it/s]

In [ ]:
import cv2
import os

input_folder = './normalMP4/SingleDistribution/'
output_file = './normalMP4/SingleDistribution.mp4'

image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])
first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, 50, (width, height))
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)
    video_writer.write(image)
video_writer.release()

print(f"Video saved as {output_file}")

In [3]:
rangeLambdas = np.concatenate([
    np.arange(0.01, 0.1, 0.005), np.arange(0.1, 1, 0.05)
])
distanceDs = np.concatenate([
    np.arange(0.1, 1, 0.05), np.arange(1, 2.1, 0.5)
])

class1, class2 = (
    np.concatenate([np.ones(500), np.zeros(500)]).astype(bool), 
    np.concatenate([np.zeros(500), np.ones(500)]).astype(bool)
)

np.random.seed(10)
omegaTheta = np.concatenate([
    np.random.uniform(1, 3, size=500),
    np.random.uniform(-3, -1, size=500)
])

In [4]:
maps = pd.read_excel("maps.xlsx", index_col=0)

globalRd = 1.2828

def lambda_c0(d0):
    rs = 1
    rd = globalRd
    if d0 + 1 < rd:
        si = 0
    else:
        beta = 2 * np.arccos((d0 ** 2 + rd ** 2 - rs ** 2) / (2 * d0 * rd))
        alpha = 2 * np.arccos((rd ** 2 + rs ** 2 - d0 ** 2) / (2 * rd * rs))
        si = alpha * rs ** 2 / 2 + beta * d0 ** 2 / 2 - d0 * rd * np.sin(beta / 2)
    Nc = 500
    deltaOmega = max(
        3 / (globalRd - 0.1) - 3 / globalRd, 
        np.abs(3 / (globalRd + 0.1) - 3 / globalRd)
    )

    NonR = Nc / 2 * (rd + d0 - 1)
    Sr1 = np.pi * ((rd + d0)**2 - 1**2)
    rho1 = NonR / Sr1
    sigmaA1 = (np.pi * d0 ** 2 - si) * rho1

    return 1 / sigmaA1 * deltaOmega

def lambda_c1(d0):
    rs = 3
    Nc = 1000 / 4
    Sr = np.pi * (3**2 - 1**2)
    deltaOmega = 2 / Nc
    beta = 2 * np.arccos(1 - d0 ** 2 / (2 * rs ** 2))
    alpha = np.pi - beta / 2
    S1 = d0**2 * alpha / 2 + rs**2 * beta / 2 - rs * d0 * np.sin(alpha / 2)
    sigmaA = S1 * Nc / Sr
    return deltaOmega / sigmaA

def lambda_c2(d0):
    rs = 1
    deltaOmega = 1
    beta = 2 * np.arccos(1 - d0 ** 2 / (2 * rs ** 2))
    alpha = np.pi - beta / 2
    N = 500 / 2
    return (
        np.pi * rs ** 2 * deltaOmega
        / (N - 1)
        / (alpha * d0**2 / 2 + beta * rs**2 / 2 - rs * d0 * np.sin(alpha / 2))
    )

def lambda_c3(d0):
    L = 10
    deltaOmega = 1
    N = 500
    rho = N / L ** 2
    sigmaA = np.pi * d0 ** 2 * rho
    return deltaOmega / sigmaA

def lambda_c4(d0):
    L = 10
    rc = 1.5
    rs = 1.5
    if d0 + 2 * (rc + rs) < L / np.sqrt(2):
        # print(d0 + 2 * (rc + rs), L / np.sqrt(2))
        return np.inf
    deltaOmega = 4
    N = 500
    rh = L / np.sqrt(2) - 2 * (rc + rs)
    rd = rh + rs
    beta = 2 * np.arccos((d0 ** 2 + rd ** 2 - rs ** 2) / (2 * d0 * rd))
    alpha = 2 * np.arccos((rd ** 2 + rs ** 2 - d0 ** 2) / (2 * rd * rs))
    si = alpha * rs ** 2 / 2 + beta * d0 ** 2 / 2 - d0 * rd * np.sin(beta / 2)
    Ni = N * si / (np.pi * rs ** 2)
    return deltaOmega / Ni

def lambda_c5(d0):
    rs = 3
    N = 1000 / 10
    rho = N / (np.pi * (3**2 - 1**2))
    deltaOmega = 2 / N
    beta = 2 * np.arccos(1 - d0 ** 2 / (2 * rs ** 2))
    alpha = np.pi - beta / 2
    Si = d0**2 * alpha / 2 + rs**2 * beta / 2 - rs * d0 * np.sin(alpha / 2)
    sigmaA = Si * rho
    return deltaOmega / sigmaA

funcIdxs = dict()
for func in [lambda_c0, lambda_c1, lambda_c2, lambda_c3, lambda_c4, lambda_c5]:
    funcIdxs[func] = []
    for d0 in maps.columns:
        l = func(d0)
        idx = 0
        upper = maps.index[maps.index > l].min()
        if np.isnan(upper):
            upper = maps.index.max()
        idx = idx + np.where(maps.index == upper)[0][0]
        lower = maps.index[maps.index < l].max()
        if np.isnan(lower):
            lower = maps.index.min()
        idx = idx + np.where(maps.index == lower)[0][0]
        funcIdxs[func].append(idx / 2)

line0 = pd.Series(funcIdxs[lambda_c0]).rolling(window=2, min_periods=1, center=True).mean().values

In [13]:
def plot_last_state_double(sa: StateAnalysis, ax: plt.Axes = None, index: int = -1,
                    showTicks: bool = False, tickSize: int = 16):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))

    singlePositionX, singlePhaseTheta, _ = sa.get_state(index=index)

    alphaRate = 0.9
    np.random.seed(10)
    omegaTheta = np.concatenate([
        np.random.uniform(1, 3, size=500),
        np.random.uniform(-3, -1, size=500)
    ])
    scale = 35
    width = 0.003
    ax.quiver(
        singlePositionX[class1, 0], singlePositionX[class1, 1],
        np.cos(singlePhaseTheta[class1]), np.sin(singlePhaseTheta[class1]), color='red', 
        alpha = (1 - alphaRate) + (np.abs(omegaTheta[class1]) - 1) / 2 * alphaRate,
        scale=scale, width=width
    )
    ax.quiver(
        singlePositionX[class2, 0], singlePositionX[class2, 1],
        np.cos(singlePhaseTheta[class2]), np.sin(singlePhaseTheta[class2]), color='#414CC7', 
        alpha = (1 - alphaRate) + (np.abs(omegaTheta[class2]) - 1) / 2 * alphaRate,
        scale=scale, width=width
    )

    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    if showTicks:
        ax.set_xticks([0, 10])
        ax.set_yticks([0, 10])
        ax.set_xlabel(r"$x$", fontsize=tickSize)
        ax.set_ylabel(r"$y$", fontsize=tickSize, rotation=0)
        ax.set_xticklabels(ax.get_xticks(), fontsize=tickSize)
        ax.set_yticklabels(ax.get_yticks(), fontsize=tickSize)
        # 拉近label与坐标轴的距离
        ax.xaxis.labelpad = -20
        ax.yaxis.labelpad = -10
    else:
        ax.set_xticks([])
        ax.set_yticks([])
    ax.grid(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['right'].set_color('black')
    plt.tick_params(direction='in', length=3)

def draw_phase_dia_double(fileName: str, mp4Path: str = "./normalMP4", step: int = 1, tailCounts: int = None):

    def plot_frame(i):
        # phase diagram
        colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
        cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        plt.plot(np.arange(maps.shape[1]), funcIdxs[lambda_c1],
                linestyle="--", color="black", lw=1, zorder = 2, label=r"$\lambda_{c1}(d_0)$")
        plt.plot(np.arange(maps.shape[1]), line0,
                linestyle="--", color="black", lw=1, zorder = 2, label=r"$\lambda_{c1}(d_0)$")
        plt.fill_between(
            np.arange(maps.shape[1]), 0, funcIdxs[lambda_c1], alpha=1, color="#403990"
        )
        plt.text(0.1, 0.3, "DS", fontsize=14, color="white", rotation=0)
        plt.plot(np.arange(maps.shape[1]), funcIdxs[lambda_c2],
                linestyle="--", color="black", lw=1, zorder = 2, label=r"$\lambda_{c1}(d_0)$")
        plt.fill_between(
            np.arange(maps.shape[1]), funcIdxs[lambda_c1], line0, alpha=1, color="#5760AE"
        )
        plt.fill_between(
            np.arange(maps.shape[1]), line0, funcIdxs[lambda_c2], alpha=1, color="#80A6E2"
        )
        plt.text(1.8, 6, "ICS", ha="center", va="center", fontsize=16, color="white")
        plt.text(4.7, 7, "CCS", ha="center", va="center", fontsize=16, color="black")
        plt.plot(np.arange(maps.shape[1]), funcIdxs[lambda_c3],
                linestyle="--", color="black", lw=1, zorder = 2, label=r"$\lambda_{c1}(d_0)$")
        plt.fill_between(
            np.arange(maps.shape[1]), funcIdxs[lambda_c2], funcIdxs[lambda_c3], alpha=1, color="#FBDD85"
        )
        plt.text(11.5, 10, "Asymptotic-clustering\nState", ha="center", va="center", fontsize=16, color="black")
        plt.plot(np.arange(maps.shape[1]), funcIdxs[lambda_c4],
                linestyle="--", color="black", lw=1, zorder = 2, label=r"$\lambda_{c1}(d_0)$")
        plt.fill_between(
            np.arange(maps.shape[1]), funcIdxs[lambda_c3], funcIdxs[lambda_c4], alpha=1, color="#F46F43"
        )
        plt.text(13, 27, "Instant-clustering\nState", ha="center", va="center", fontsize=16, color="black")
        plt.fill_between(
            np.arange(maps.shape[1]), funcIdxs[lambda_c4], 35, alpha=1, color="#CF3D3E"
        )
        plt.text(18.8, 20, "Instant Sync State", fontsize=16, color="Black", rotation=-90)
        xIdxs = np.arange(len(distanceDs), step=5)
        xIdxs = np.insert(xIdxs, -1, 18)
        plt.xticks(xIdxs, np.round(distanceDs, 2)[xIdxs])
        yIdxs = np.concatenate([np.arange(19, step=6), 18 + np.arange(18, step=6)[1:], [35]])
        idx = np.where(rangeLambdas == 0.1)[0][0]
        yIdxs = np.insert(yIdxs, 4, idx)
        plt.yticks(yIdxs, np.round(rangeLambdas, 2)[yIdxs])
        plt.tick_params(direction='in', length=3)

        plt.xlim(0, 20)
        plt.ylim(0, 35)
        plt.xlabel(r"$d_0$")
        plt.ylabel(r"$\lambda$", rotation=0)
        plt.grid(False)
        # snapshot
        axSwarmM = inset_axes(ax, width="40%", height="40%", loc='lower left',
                            bbox_to_anchor=(-0.8, 0.5, 1.5, 1.5),
                            bbox_transform=ax.transAxes)
        ax.plot([-4, 4], [30, 24], clip_on=False, color="black", lw=1)
        plot_last_state_double(sas[0], ax=axSwarmM, index=i)
        axSwarmM.set_title(r"$\mathbf{(b)}\ \lambda=0.4,\ d_0=0.3$")

        axRing = inset_axes(ax, width="40%", height="40%", loc='lower left',
                            bbox_to_anchor=(-0.8, -0.2, 1.5, 1.5),
                            bbox_transform=ax.transAxes)
        ax.plot([-4, 6], [5, 2], clip_on=False, color="black", lw=1)
        plot_last_state_double(sas[1], ax=axRing, index=i)
        axRing.set_title(r"$\mathbf{(c)}\ \lambda=0.02,\ d_0=0.4$")

        axQuickSync = inset_axes(ax, width="40%", height="40%", loc='lower left',
                                bbox_to_anchor=(1.1, 0.5, 1.5, 1.5),
                                bbox_transform=ax.transAxes)
        ax.plot([25, 20], [30, 35], clip_on=False, color="black", lw=1)
        plot_last_state_double(sas[2], ax=axQuickSync, index=i)
        axQuickSync.set_title(r"$\mathbf{(d)}\ \lambda=0.95,\ d_0=2$")

        axSwarmD = inset_axes(ax, width="40%", height="40%", loc='lower left',
                            bbox_to_anchor=(1.1, -0.2, 1.5, 1.5),
                            bbox_transform=ax.transAxes)
        ax.plot([25, 16], [3, 19], clip_on=False, color="black", lw=1)
        plot_last_state_double(sas[3], ax=axSwarmD, index=i)
        axSwarmD.set_title(r"$\mathbf{(e)}\ \lambda=0.15,\ d_0=0.9$")

        colorbarAx1 = inset_axes(ax, width="2%", height="40%", loc='lower left',
                                bbox_to_anchor=(1.75, 0.5, 1.5, 1.5),
                                bbox_transform=ax.transAxes)

        colors = ["#FFE6E6", "#FF4B4E"]
        rcmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
        sca = ax.scatter(-np.arange(omegaTheta.size), -np.ones_like(omegaTheta), 
                c=np.linspace(1, 3, len(omegaTheta)), cmap=rcmap)
        cbar = plt.colorbar(sca, cax=colorbarAx1)
        cbar.set_label(r"$\omega>0$", fontsize=14, labelpad=-39)
        cbar.ax.tick_params(direction='in', length=3)

        colorbarAx2 = inset_axes(ax, width="2%", height="40%", loc='lower left',
                                bbox_to_anchor=(1.75, -0.2, 1.5, 1.5),
                                bbox_transform=ax.transAxes)
        colors = ["#414CC7", "#E6E6FF"]
        rcmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
        sca = ax.scatter(-np.arange(omegaTheta.size), -np.ones_like(omegaTheta), 
                c=-np.linspace(1, 3, len(omegaTheta)), cmap=rcmap)
        cbar = plt.colorbar(sca, cax=colorbarAx2)
        cbar.set_label(r"$\omega<0$", fontsize=14, labelpad=-47)
        cbar.ax.tick_params(direction='in', length=3)

        ax.set_title(r"$\mathbf{(a)}$", loc="left", fontsize=18)

        plt.savefig(f"{mp4Path}/{fileName}/{i}.png", dpi=100, bbox_inches="tight")
        plt.close()

    models = [
        SpatialGroups(0.4, 0.3, savePath="./data", randomSeed=randomSeed, tqdm=True, overWrite=True),
        SpatialGroups(0.02, 0.4, savePath="./data", randomSeed=80, tqdm=True, overWrite=True),
        SpatialGroups(0.95, 2, savePath="./data", randomSeed=randomSeed, tqdm=True, overWrite=True),
        SpatialGroups(0.15, 0.9, savePath="./data", randomSeed=randomSeed, tqdm=True, overWrite=True)
    ]
    np.random.seed(10)
    omegaTheta = np.random.uniform(1, 3, size=500)

    sas = [StateAnalysis(model, showTqdm=False) for model in models]

    # fig, ax = plt.subplots(1, 1, figsize=(5.1, 5))
    TNum = sas[0].TNum
    startIdx = TNum - tailCounts * step if tailCounts else 0
    frames = np.arange(startIdx, TNum, step)
    pbar = tqdm(total=len(frames))
    for i in frames:
        plot_frame(i)
        pbar.update(1)
    pbar.close()

In [ ]:
draw_phase_dia_double("SpatialGroups", step=1, tailCounts=750)

In [ ]:
import cv2
import os

input_folder = './normalMP4/SpatialGroups/'
output_file = './normalMP4/SpatialGroups.mp4'

image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])
first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, 50, (width, height))
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)
    video_writer.write(image)
video_writer.release()

print(f"Video saved as {output_file}")

Video saved as ./normalMP4/SpatialGroups.mp4
